<a href="https://colab.research.google.com/github/jkordonis/TropicalFactorization/blob/main/MovieLensTropFactorization_HitRate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import cupy as np
#import numpy as np


In [ ]:
# @title Load the MovieLens data (run this cell).

# Download MovieLens data.
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfilea

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')





Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [ ]:
indices_pd = ratings[['user_id', 'movie_id']].values
IND=np.array(indices_pd).astype(int)
IND_x,IND_y = IND[:,0],IND[:,1]

n = 944#(np.max(IND,0)[0]+1).astype(int)
p = 1683# (np.max(IND,0)[1]+1).astype(int)
r = 40
B=np.zeros((n,p))+1
m=20

B[IND_x,IND_y]=-1

In [ ]:
pairs=[(i,j) for j in range(p) for i in range(n)]
pairs=np.array(pairs)
np.random.shuffle(pairs)

N=1271000 #n*p*0.8
N_=1429876
N__=n*p


In [ ]:
IND_x_train = pairs[:N,0]
IND_y_train = pairs[:N,1]


IND_x_val  = pairs[N:N_,0]
IND_y_val  = pairs[N:N_,1]

IND_x_tst  = pairs[N_:N__,0]
IND_y_tst  = pairs[N_:N__,1]

B_sparse_ob_or =  B[IND_x_train,IND_y_train]
B_sparse_ob_or_val =  B[IND_x_val,IND_y_val]
B_sparse_ob_or_tst = B[IND_x_tst,IND_y_tst]


In [ ]:
 # Initialize C,D matrix estimates
C_0,D_Tr_0=np.random.rand(n,r),np.random.rand(p,r)


In [ ]:
C=np.copy(C_0)
D_Tr=np.copy(D_Tr_0)
Val_err=1000000

In [ ]:
C=np.copy(C_0)
D_Tr=np.copy(D_Tr_0)
#C=np.copy(C_op)
#D_Tr = np.copy(D_op)

import time
T=time.time()
#cnt=0
Err =np.zeros(100000)
Err_val =np.zeros(100000)
Err_tst =np.zeros(100000)

m=20
for _ in range(50):
  for _ in range(1):
    INDICES=np.array(range(N))
    np.random.shuffle(INDICES)
    BatchSize=1000
    i=0
    minibatches=[]
    while i*BatchSize<=N:
      minibatches.append(INDICES[i*BatchSize:BatchSize*(i+1)])
      i+=1

    #minibatches = [INDICES[i*BatchSize:BatchSize*(i+1)] for i in range(np.ceil(N/BatchSize).astype(int))]
    for AA in minibatches:
      IND_x_ob=IND_x_train[AA]
      IND_y_ob=IND_y_train[AA]
      B_sparse_ob=B_sparse_ob_or[AA]

      Maximizers = np.argmax(C[IND_x_ob]+D_Tr[IND_y_ob],-1)
      B_sparse_ob_est = C[IND_x_ob,Maximizers]+D_Tr[IND_y_ob,Maximizers]
      Differences = B_sparse_ob_est -B_sparse_ob
    #  for i in range(IND_x_ob.shape[0]):
      C_to_change = C[IND_x_ob,Maximizers]
      D_to_change = D_Tr[IND_y_ob,Maximizers]

      C[IND_x_ob,Maximizers]=(C_to_change-0.03*Differences)#*0.9998
      D_Tr[IND_y_ob,Maximizers]=(D_to_change-0.03*Differences)#*0.9998


    Maximizers = np.argmax(C[IND_x_train]+D_Tr[IND_y_train],-1)
    B_sparse_ob_est = C[IND_x_train,Maximizers]+D_Tr[IND_y_train,Maximizers]
    B_sparse_ob=B_sparse_ob_or[range(N)]
    Differences = B_sparse_ob_est -B_sparse_ob
    Err[cnt]=np.linalg.norm(Differences)
    #C=C+0.00031
    #D_Tr=D_Tr+0.00031

    Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
    B_sparse_ob_est = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
    Differences_tst = B_sparse_ob_est -B_sparse_ob_or_tst

    Maximizers = np.argmax(C[IND_x_val]+D_Tr[IND_y_val],-1)
    B_sparse_ob_est = C[IND_x_val,Maximizers]+D_Tr[IND_y_val,Maximizers]
    Differences_val = B_sparse_ob_est -B_sparse_ob_or_val
    Err_val[cnt]=np.linalg.norm(Differences_val)
    Err_tst[cnt]=np.linalg.norm(Differences_tst)
    if Val_err>Err_val[cnt]:
      Val_err=Err_val[cnt]
      C_op = np.copy(C)
      D_op = np.copy(D_Tr)

    cnt+=1
    U, s, Vt = np.linalg.svd(D_Tr, full_matrices=False)
    Diag_sk_sqrt=np.diag(s[:m]**0.5)
    Uk = U[:, :m]@Diag_sk_sqrt
    Vk = Diag_sk_sqrt@Vt[:m, :]
    D_Tr = Uk@Vk #make comment for simple tropical uncomment for mixed


  print(time.time()-T,Err[cnt-1],np.linalg.norm(Differences_val),np.linalg.norm(Differences_tst))

1.4251039028167725 510.1407567058731 179.9393582372695 178.93300339720759
2.8189499378204346 491.734926943941 174.1136231233614 173.08074802552423
4.1811065673828125 479.76880373284496 170.36884266888953 169.3666001912582
5.5435791015625 471.8927803109653 167.8576776316467 166.97441646023745
6.915744781494141 465.06338815179294 165.79575119722588 164.93298983964374
8.271004676818848 459.2618074250704 164.18618101751684 163.33837196467744
9.685354948043823 454.34834012252054 162.74055327741922 162.07333044536588
11.060433864593506 450.4961328562543 161.87701772797018 161.03634265614565
12.411330223083496 447.0805410093467 161.0214232223535 160.33087559681994
13.790696382522583 443.91883656482935 160.19439576946158 159.55459961979557
15.153501272201538 441.10238479343525 159.64898716149355 159.0777730734191
16.528454780578613 438.77781274218364 159.32859025834546 158.6147963284408
17.912556886672974 436.6610009062746 158.86705539137984 158.2785270108404
19.29612135887146 434.515939706613

In [ ]:
C=np.copy(C_op)
D_Tr = np.copy(D_op)
Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
B_sparse_ob_est = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
Differences_tst = B_sparse_ob_est -B_sparse_ob_or_tst
np.linalg.norm(Differences_tst/((N_-N)**0.5))

array(0.39177143)

In [ ]:
indexmin=np.argmin(    Err_val[:cnt])
print(Err_tst[indexmin]/((N_-N)**0.5))

0.0


In [ ]:
Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
B_sparse_ob_est_tst = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
B_tst = B[IND_x_tst,IND_y_tst]
np.linalg.norm(B_sparse_ob_est_tst-B_tst)

Precision_ = 0
Precision_rand=0
for i in range(n):
  AAA=(i==IND_x_tst)
  x_ind = IND_x_tst[AAA]
  y_ind = IND_y_tst[AAA]
  ACTUAL_score = B[x_ind,y_ind]
  Estimated=B_sparse_ob_est_tst[AAA]
  Precision_+= (ACTUAL_score[np.argsort(Estimated)[:10]]==-1).sum()
  Precision_rand+= (ACTUAL_score[np.random.randint(0,ACTUAL_score.shape[0],10)]==-1).sum()

print('Precision@10=',Precision_/(10*n))
print('Precision_@10_RANDOMchoice=',Precision_rand/(10*n))



Precision@10= 0.40953389830508474
Precision_@10_RANDOMchoice= 0.06038135593220339


In [ ]:
HR_mat=np.zeros(100)
for epan in range(100):
  count=0
  HIT=0
  for i in range(1,n):
    AAA=(i==IND_x_tst)
    x_ind = IND_x_tst[AAA]
    y_ind = IND_y_tst[AAA]
    ACTUAL_score = B[x_ind,y_ind]
    Estimated=B_sparse_ob_est_tst[AAA]

    AA=np.array(range(ACTUAL_score.shape[0]))
    Positive_all_ind=np.array(ACTUAL_score==-1)
    Positive_all=AA[Positive_all_ind]
    if Positive_all.shape[0]>0:
      Positive_sample = Positive_all[np.random.randint(0,Positive_all.shape[0])]
      Negative_all_ind=np.array(ACTUAL_score==1)
      Negative_all=AA[Negative_all_ind]
      Negative_sample_Ind_of_ind=np.random.choice(Negative_all.shape[0], 100)
      Negative_samples = Negative_all[Negative_sample_Ind_of_ind]
      Samples = np.concatenate((np.array([Positive_sample]),Negative_samples), axis=0)
      count+=1
      HIT+=(Estimated[Samples]<Estimated[Samples[0]]).sum() <=10
  HR_mat[epan]=HIT/count

  HR_mat.mean()

In [ ]:
HR_mat.mean()

array(0.75777174)

NameError: ignored